SparkML


In [8]:
import os
os.environ["JAVA_HOME"] = "C:/Javaa"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]

In [9]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import OneHotEncoder, StringIndexer, StandardScaler, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import rand
from pyspark.mllib.evaluation import MulticlassMetrics

In [10]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras import optimizers, regularizers
from keras.optimizers import Adam

In [11]:
from elephas.ml_model import ElephasEstimator

WARNING


In [236]:
import winsound
# Play Windows exit sound.

In [12]:
conf = SparkConf().setAppName('Spark DL Tabular Pipeline').setMaster('local[6]')

In [13]:
sc = SparkContext(conf=conf)
sql_context = SQLContext(sc)

In [3]:
import pandas as pd 
import numpy as np
import sqlite3

In [4]:
con = sqlite3.connect(r"C:\Users\reekithak\Downloads\drugs.db")

In [19]:
df = pd.read_sql("SELECT * from train_set0",con)

In [68]:
temp_ls = ["emb{}".format(s) for s in range(0,32)]

In [41]:
#exploding the Dataframe

new_data = pd.concat([pd.Series(row['cid']                      
                                ,row['target'].split(",")) for _,row in df.iterrows()]).reset_index()

In [122]:
new_data.columns

Index(['index', 0], dtype='object')

In [230]:
df.shape

(509983, 34)

In [227]:
#Exploding the data fro efficiency

In [ ]:
cid_list = list(df['cid'].unique())
new_df = pd.DataFrame()
iter_ = 0
num_=1
for cid in cid_list:
    #print(num_)
    if(iter_==500):
        print(cid)
        iter_=0
    elif(iter_==50000):
        iter_=0
        print("50k")
        
    else:
        pass
    temp_df = new_data.loc[new_data[0]==str(cid)].reset_index()
    temp_mix_df = df.loc[df['cid']==str(cid)][['cid','emb0',
 'emb1',
 'emb2',
 'emb3',
 'emb4',
 'emb5',
 'emb6',
 'emb7',
 'emb8',
 'emb9',
 'emb10',
 'emb11',
 'emb12',
 'emb13',
 'emb14',
 'emb15',
 'emb16',
 'emb17',
 'emb18',
 'emb19',
 'emb20',
 'emb21',
 'emb22',
 'emb23',
 'emb24',
 'emb25',
 'emb26',
 'emb27',
 'emb28',
 'emb29',
 'emb30',
 'emb31']].reset_index()
    interdf = pd.DataFrame()
    interdf = interdf.append([temp_mix_df]*temp_df.shape[0],ignore_index=True)
    new_temp  = pd.concat([temp_df,interdf],axis=1)
    new_df = new_df.append(new_temp)
    try:
        new_df.drop('level_0',axis=1,inplace=True)
    except:
        pass
    new_df.reset_index(inplace=True)
    iter_+=1
    num_+=1
new_df.to_csv("train_set0_new.csv")

**RUN THE AABOVE FOR COMPLETE GRAPH**

In [308]:
new_df = new_df.drop(['level_0'],axis=1)

In [310]:
new_df =  new_df.drop(new_df.iloc[:,1:2],axis=1)

In [312]:
cols = []
count = 1 
for column in new_df.columns:
    if column=='index':
        cols.append(f'index_{count}')
        count+=1
        continue
    cols.append(column)

In [313]:
new_df.columns = cols
new_df = new_df.drop(['index_2'],axis=1)

In [316]:
new_df['target'] = new_df['index_1'].tolist()

In [318]:
new_df = new_df.drop(['index_1'],axis=1)

In [319]:
new_df

,cid,emb0,emb1,emb2,emb3,emb4,emb5,emb6,emb7,emb8,...,emb23,emb24,emb25,emb26,emb27,emb28,emb29,emb30,emb31,target
0,3081361,4.719912,-1.287492,5.796409,3.760270,-2.509900,-3.106354,6.347282,3.284466,-5.520523,...,-8.252186,-0.474333,5.553354,-3.007781,-6.632124,-2.022637,-5.149702,-5.259633,-5.783206,1D4H
1,3081361,4.719912,-1.287492,5.796409,3.760270,-2.509900,-3.106354,6.347282,3.284466,-5.520523,...,-8.252186,-0.474333,5.553354,-3.007781,-6.632124,-2.022637,-5.149702,-5.259633,-5.783206,1FLT
2,3081361,4.719912,-1.287492,5.796409,3.760270,-2.509900,-3.106354,6.347282,3.284466,-5.520523,...,-8.252186,-0.474333,5.553354,-3.007781,-6.632124,-2.022637,-5.149702,-5.259633,-5.783206,1Y6A
3,3081361,4.719912,-1.287492,5.796409,3.760270,-2.509900,-3.106354,6.347282,3.284466,-5.520523,...,-8.252186,-0.474333,5.553354,-3.007781,-6.632124,-2.022637,-5.149702,-5.259633,-5.783206,1EVT
4,3081361,4.719912,-1.287492,5.796409,3.760270,-2.509900,-3.106354,6.347282,3.284466,-5.520523,...,-8.252186,-0.474333,5.553354,-3.007781,-6.632124,-2.022637,-5.149702,-5.259633,-5.783206,1IVO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1504,461021,0.447797,-0.300073,0.475092,1.214543,-0.731829,0.065035,-0.286810,-0.513443,0.891044,...,0.258407,-0.056168,-0.329731,0.972165,-0.741493,-0.022831,0.417377,-0.044310,0.096669,1EBY
1505,461022,0.403545,-0.174842,0.431639,1.103528,-0.649381,-0.068627,-0.403820,-0.652268,0.834180,...,0.266108,-0.060752,-0.169403,0.915691,-0.574934,0.092033,0.149672,0.089487,0.155100,1EBY
1506,461023,0.316273,-0.042992,0.444821,1.055882,-0.743242,0.234285,-0.145193,-0.226776,0.983365,...,0.335365,-0.117928,0.178858,1.285986,-0.570744,-0.316009,0.434754,0.007619,0.252445,1EBY
1507,461024,0.252075,-0.201987,0.447130,1.105489,-0.835660,-0.135988,-0.531263,-0.463336,0.987653,...,0.221171,0.109694,0.128133,0.958728,-0.552585,-0.028586,0.286612,-0.204293,0.281764,1EBY


In [320]:
sc

<SparkContext master=local[6] appName=Spark DL Tabular Pipeline>

In [16]:
df = sql_context.read.csv('train_set0.csv',
                                header=True,
                                inferSchema=True)

In [17]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- cid: integer (nullable = true)
 |-- emb0: double (nullable = true)
 |-- emb1: double (nullable = true)
 |-- emb2: double (nullable = true)
 |-- emb3: double (nullable = true)
 |-- emb4: double (nullable = true)
 |-- emb5: double (nullable = true)
 |-- emb6: double (nullable = true)
 |-- emb7: double (nullable = true)
 |-- emb8: double (nullable = true)
 |-- emb9: double (nullable = true)
 |-- emb10: double (nullable = true)
 |-- emb11: double (nullable = true)
 |-- emb12: double (nullable = true)
 |-- emb13: double (nullable = true)
 |-- emb14: double (nullable = true)
 |-- emb15: double (nullable = true)
 |-- emb16: double (nullable = true)
 |-- emb17: double (nullable = true)
 |-- emb18: double (nullable = true)
 |-- emb19: double (nullable = true)
 |-- emb20: double (nullable = true)
 |-- emb21: double (nullable = true)
 |-- emb22: double (nullable = true)
 |-- emb23: double (nullable = true)
 |-- emb24: double (nullable = true)
 |-- emb

In [18]:
df.limit(5).toPandas()

,_c0,cid,emb0,emb1,emb2,emb3,emb4,emb5,emb6,emb7,...,emb23,emb24,emb25,emb26,emb27,emb28,emb29,emb30,emb31,target
0,0,3081361,4.719912,-1.287492,5.796409,3.760270,-2.509900,-3.106354,6.347282,3.284466,...,-8.252186,-0.474333,5.553354,-3.007781,-6.632124,-2.022637,-5.149702,-5.259633,-5.783206,"1D4H,1FLT,1Y6A,1EVT,1IVO,1Y6A,1FLT,1JU5,2XB7,2..."
1,1,5327236,1.248603,-0.253252,-0.254269,0.804871,-0.980929,-0.430769,0.525774,-0.733890,...,-0.211978,-0.483960,-1.107684,0.974848,-0.160632,2.125608,0.882338,-1.016395,1.250610,"1D4H,1D4K"
2,2,5327235,1.409250,-0.664244,-0.052777,1.251166,-0.900441,-0.416737,0.396938,-1.056493,...,-0.250652,-0.127260,-1.264243,1.083983,-0.324660,2.211021,0.841747,-0.969820,1.200209,"1D4H,1D4K"
3,3,5327234,0.862983,-0.649309,0.103232,0.856554,-0.948067,-0.355748,-0.125225,-0.140954,...,0.018531,-0.510167,-0.576433,1.177396,-0.220560,0.872600,0.954063,-0.638187,0.492565,1D4H
4,4,3009319,1.577372,-0.625860,0.031868,0.781114,-1.194058,-0.452654,0.164674,-0.567973,...,-0.213216,-0.307816,-1.132520,0.917929,-0.634182,2.012652,1.027961,-0.934430,1.287832,"1D4H,1D4K"
